In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>, Jongho Lee <jongho.lee@cern.ch>, Cristovao Beirao da Cruz e Sliva <cberiaod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

## Import

In [ ]:
import beamtest_analysis_helper as helper
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from natsort import natsorted
import hist
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data

today = datetime.date.today().isoformat()
fig_outdir = Path('../../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)


chip_labels = [0, 1, 2, 3]
chip_names = ["ET2_EPIR_Pair1", "ET2_BAR_4", "ET2_BAR_6", "ET2_CNM_1-3"]
offsets = [15, 6, 15, 6]
high_voltages = [260, 260, 260, 200]

chip_fignames = chip_names
chip_figtitles = [
    f"(Trigger) Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"(DUT1) Bar4 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"(Reference) Bar6 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"(DUT2) CNM (HPK) 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"]

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
plt.show()

## Loading Pandas dataframe and filter events with a single hit per board

In [ ]:
files = glob('./desy_TB_run12/Run_12_loop_[1-9].feather')

last_evt = 0
dataframes = []

for idx, ifile in enumerate(files):
    tmp_df = pd.read_feather(ifile)
    tmp_df.drop(columns=['evt_number', 'bcid', 'l1a_counter', 'ea'], inplace=True)

    if idx > 0:
        tmp_df['evt'] += last_evt
    last_evt += tmp_df['evt'].unique()[-1]


    ## Selecting good hits
    tdc_cuts = {
        # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
        0: [tmp_df.loc[tmp_df['board'] == 0]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == 0]['cal'].mode()[0]+50, 100, 500,  0, 600],
        1: [tmp_df.loc[tmp_df['board'] == 1]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == 1]['cal'].mode()[0]+50,   0, 1100, 0, 600],
        2: [tmp_df.loc[tmp_df['board'] == 2]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == 2]['cal'].mode()[0]+50,   0, 1100, 0, 600],
        3: [tmp_df.loc[tmp_df['board'] == 3]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == 3]['cal'].mode()[0]+50,   0, 1100, 0, 600],
    }

    filtered_df = helper.tdc_event_selection(tmp_df, tdc_cuts_dict=tdc_cuts)
    del tmp_df

    if filtered_df.empty:
        continue

    event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
    event_selection_col = None

    trig_selection = (event_board_counts[0] == 1)
    ref_selection = (event_board_counts[2] == 1)
    event_selection_col = trig_selection & ref_selection

    selected_event_numbers = event_board_counts[event_selection_col].index
    selected_subset_df = filtered_df[filtered_df['evt'].isin(selected_event_numbers)]
    selected_subset_df.reset_index(inplace=True, drop=True)

    dataframes.append(selected_subset_df)
    del event_board_counts, selected_event_numbers, selected_subset_df, event_selection_col

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes

In [ ]:
df.info()

In [ ]:
h_inclusive = helper.return_hist(df, chip_names, chip_labels, hist_bins=[100, 128, 128])

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[3], chip_fignames[3], chip_figtitles[3], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

del h_inclusive

In [ ]:
helper.plot_heatmap_byPandas(df, chipLabels=chip_labels, figtitle=chip_figtitles, figtitle_tag='')

## Efficiency studies

### Select a one pixel of a trigger board, and make occupancy map of other boards

In [ ]:
eff_df = helper.efficiency_with_single_board(df, pixel=(7, 5), board_id=3)
helper.plot_heatmap_byPandas(eff_df, chip_labels, chip_figtitles, "inclusive")

In [ ]:
eff_df = helper.efficiency_with_two_boards(df, pixel=(7, 5), board_ids=(0, 2))
helper.plot_heatmap_byPandas(eff_df, chip_labels, chip_figtitles, "inclusive")

In [ ]:
h_inclusive = helper.return_hist(eff_df, chip_names, chip_labels, hist_bins=[100, 256, 512])

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_inclusive, chip_names[3], chip_fignames[3], chip_figtitles[3], fig_path, save=False, show=True,
                                tag="inclusive", title_tag=", inclusive", slide_friendly=True)

del h_inclusive

## Event counter based on possible track combination

In [ ]:
single_filtered_df = helper.singlehit_event_clear(df)
pivot_data_df = helper.making_pivot(single_filtered_df, 'evt', 'board', set({'board', 'evt', 'cal', 'tot'}), ignore_boards=[3])
# pivot_data_df = helper.making_pivot(df, 'evt', 'board', set({'board', 'evt', 'cal', 'tot'}), ignore_boards=[2])
del single_filtered_df

In [ ]:
min_hit_counter = 1200
# combinations_df = pivot_data_df.groupby(['row_0', 'col_0', 'row_1', 'col_1', 'row_2', 'col_2', 'row_3', 'col_3', ]).count()
combinations_df = pivot_data_df.groupby(['row_0', 'col_0', 'row_1', 'col_1', 'row_2', 'col_2', ]).count()
combinations_df['count'] = combinations_df['toa_0']
# combinations_df.drop(['toa_0', 'toa_1', 'toa_2', 'toa_3'], axis=1, inplace=True)
combinations_df.drop(['toa_0', 'toa_1', 'toa_2'], axis=1, inplace=True)
track_df = combinations_df.loc[combinations_df['count'] > min_hit_counter]
track_df.reset_index(inplace=True)
del combinations_df
# del pivot_data_df, combinations_df

In [ ]:
a = (0.5*(track_df['row_0'] + track_df['row_2']) - track_df['row_1'])**2
b = (0.5*(track_df['col_0'] + track_df['col_2']) - track_df['col_1'])**2
track_selection = (np.sqrt(a+b) <= 2.)

In [ ]:
track_df[track_selection]

In [ ]:
df

In [ ]:
## --------------------------------------
def new_pixel_filter(
        input_df: pd.DataFrame,
        pixel_dict: dict
    ):
    # Create boolean masks for each board's filtering criteria
    masks = {}
    for board, pix in pixel_dict.items():
        mask = (
            (input_df['board'] == board) & (input_df['row'] == pix[0]) & (input_df['col'] == pix[1])
        )
        masks[board] = mask

    # Combine the masks using logical All
    combined_mask = pd.concat(masks, axis=1).any(axis=1)

    # Apply the combined mask to the DataFrame
    filtered = input_df[combined_mask].reset_index(drop=True)
    return filtered

In [ ]:
mask1 = (df['board'] == 0) & (df['row'] == 0) & (df['col'] == 4)
mask2 = (df['board'] == 1) & (df['row'] == 0) & (df['col'] == 4)
mask3 = (df['board'] == 2) & (df['row'] == 0) & (df['col'] == 4)

masks = {
    0: mask1,
    1: mask2,
    2: mask3,
}

combined_mask = pd.concat(masks, axis=1)
# combined_mask[(combined_mask[1] == True)]
a = (combined_mask[0] == True) | (combined_mask[1] == True) | (combined_mask[2] == True)

In [ ]:
pix_dict = {
    # board ID: [row, col]
    0: [ 0, 4],
    1: [ 0, 4],
    2: [ 0, 4],
}

filtered_group = new_pixel_filter(df, pix_dict)
filtered_group

In [ ]:
board_to_analyze = [0, 1, 2]
track_pivots = []

for i in range(len(track_df[track_selection])):

    pix_dict = {}
    for idx in board_to_analyze:
        pix_dict[idx] = [track_df.iloc[i][f'row_{idx}'], track_df.iloc[i][f'col_{idx}']]

    filtered_group = helper.pixel_filter(df, pix_dict)
    print(filtered_group)

    # pivot_table = filtered_group.pivot(index=["evt"], columns=["board"], values=["row", "col", "toa", "tot", "cal"])
    # track_pivots.append(pivot_table)

    break

In [ ]:
track_pivots[0]

In [ ]:
pix_dict = {
    # board ID: [row, col]
    0: [ 9, 5],
    1: [ 9, 5],
    2: [ 9, 5],
    3: [ -1, -1],
}

filtered_group = helper.pixel_filter(df, pix_dict)
filtered_group.info()

## Bootstrap algorithm to calculate time resolution

In [ ]:
track_pivots = []

for i in range(len(track_df)):

    pix_dict = {}
    for idx in board_to_analyze:
        pix_dict[idx] = [track_df.iloc[i][f'row_{idx}'], track_df.iloc[i][f'col_{idx}']]

    filtered_group = helper.pixel_filter(df, pix_dict)
    filtered_group = helper.singlehit_event_clear_func(filtered_group)
    pivot_table = filtered_group.pivot(index=["evt"], columns=["board"], values=["row", "col", "toa", "tot", "cal"])
    track_pivots.append(pivot_table)

In [ ]:
track_pivots[0]['row'][0].unique()[0]

In [ ]:
from tqdm import tqdm

final_dict = {}

for idx in board_to_analyze:
    final_dict[f'row{idx}'] = []
    final_dict[f'col{idx}'] = []
    final_dict[f'res{idx}'] = []
    final_dict[f'err{idx}'] = []

for itable in tqdm(track_pivots):

    sum_arr = {}
    sum_square_arr = {}
    iteration = 100
    sampling_fraction = 0.75
    counter = 0

    for idx in board_to_analyze:
        sum_arr[idx] = 0
        sum_square_arr[idx] = 0

    for iloop in range(iteration):

        try_df = itable.reset_index()
        tdc_cuts = {}
        for idx in board_to_analyze:
            # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
            if idx == 0:
                tdc_cuts[idx] = [try_df['cal'][idx].mean()-5, try_df['cal'][idx].mean()+5,  350, 500, 0, 600]
            else:
                tdc_cuts[idx] = [try_df['cal'][idx].mean()-5, try_df['cal'][idx].mean()+5,  0, 1100, 0, 600]

        tdc_filtered_df = helper.tdc_event_selection_pivot(try_df, tdc_cuts)
        del try_df, tdc_cuts

        n = int(sampling_fraction*tdc_filtered_df.shape[0])
        indices = np.random.choice(tdc_filtered_df['evt'].unique(), n, replace=False)
        tdc_filtered_df = tdc_filtered_df.loc[tdc_filtered_df['evt'].isin(indices)]

        if tdc_filtered_df.shape[0] < iteration/(3.*(1-sampling_fraction)):
            print('Warning!! Sampling size is too small. Skipping this track')
            break

        d = {
            'evt': tdc_filtered_df['evt'].unique(),
        }

        for idx in board_to_analyze:
            bins = 3.125/tdc_filtered_df['cal'][idx].mean()
            d[f'toa_b{str(idx)}'] = 12.5 - tdc_filtered_df['toa'][idx] * bins
            d[f'tot_b{str(idx)}'] = (2*tdc_filtered_df['tot'][idx] - np.floor(tdc_filtered_df['tot'][idx]/32)) * bins

        df_in_time = pd.DataFrame(data=d)
        del d, tdc_filtered_df

        corr_toas = helper.three_board_iterative_timewalk_correction(df_in_time, 5, 3, board_list=board_to_analyze)

        diffs = {}
        for board_a in board_to_analyze:
            for board_b in board_to_analyze:
                if board_b <= board_a:
                    continue
                name = f"{board_a}{board_b}"
                diffs[name] = np.asarray(corr_toas[f'toa_b{board_a}'] - corr_toas[f'toa_b{board_b}'])

        hists = {}
        for key in diffs.keys():
            hists[key] = hist.Hist(hist.axis.Regular(80, -1.2, 1.2, name="TWC_delta_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
            hists[key].fill(diffs[key])

        try:
            fit_params_lmfit = {}
            for key in hists.keys():
                params = helper.lmfit_gaussfit_with_pulls(diffs[key], hists[key], std_range_cut=0.4, width_factor=1.25, fig_title='',
                                                    use_pred_uncert=True, no_show_fit=False, no_draw=True, get_chisqure=False)
                fit_params_lmfit[key] = params
            del params, hists, diffs, corr_toas

            resolutions = helper.return_resolution_three_board(fit_params_lmfit, var=list(fit_params_lmfit.keys()), board_list=board_to_analyze)

            if any(np.isnan(val) for key, val in resolutions.items()):
                print('fit results is not good, skipping this iteration')
                continue

            for key in resolutions.keys():
                sum_arr[key] += resolutions[key]
                sum_square_arr[key] += resolutions[key]**2

            counter += 1

        except:
            print('Failed, skipping')
            del hists, diffs, corr_toas

    if counter != 0:
        for idx in board_to_analyze:
            final_dict[f'row{idx}'].append(itable['row'][idx].unique()[0])
            final_dict[f'col{idx}'].append(itable['col'][idx].unique()[0])

        for key in sum_arr.keys():
            mean = sum_arr[key]/counter
            std = np.sqrt((1/(counter-1))*(sum_square_arr[key]-counter*(mean**2)))
            final_dict[f'res{key}'].append(mean)
            final_dict[f'err{key}'].append(std)
    else:
        print('Track is not validate for bootstrapping')

In [ ]:
final_df = pd.DataFrame(data=final_dict)
final_df

### Pixel ID selection based on the event counter

In [ ]:
pix_dict = {
    # board ID: [row, col]
    0: [ 9, 5],
    1: [ 9, 5],
    2: [ 9, 5],
    3: [ -1, -1],
}

filtered_group = helper.pixel_filter(df, pix_dict)
filtered_group = helper.singlehit_event_clear(filtered_group)
filtered_group.info()

In [ ]:
del df

In [ ]:
h_pix_selected = helper.return_hist(filtered_group, chip_names, chip_labels, hist_bins=[100, 128, 256])

helper.plot_1d_TDC_histograms(h_pix_selected, chip_names[0], chip_fignames[0], chip_figtitles[0],
                                fig_path, save=False, show=True, tag="inclusive",
                                title_tag=f", Pixel ({pix_dict[0][0]}, {pix_dict[0][1]})", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_pix_selected, chip_names[1], chip_fignames[1], chip_figtitles[1],
                                fig_path, save=False, show=True, tag="inclusive",
                                title_tag=f", Pixel ({pix_dict[1][0]}, {pix_dict[1][1]})", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_pix_selected, chip_names[2], chip_fignames[2], chip_figtitles[2],
                                fig_path, save=False, show=True, tag="inclusive",
                                title_tag=f", Pixel ({pix_dict[2][0]}, {pix_dict[2][1]})", slide_friendly=True)

helper.plot_1d_TDC_histograms(h_pix_selected, chip_names[3], chip_fignames[3], chip_figtitles[3],
                                fig_path, save=False, show=True, tag="inclusive",
                                title_tag=f", Pixel ({pix_dict[3][0]}, {pix_dict[3][1]})", slide_friendly=True)

del h_pix_selected

In [ ]:
filtered_group

### TDC cut

In [ ]:
# Define custom filtering criteria for each board
tdc_cuts = {
    # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
    0: [filtered_group.loc[filtered_group['board'] == 0]['cal'].mode()[0]-3, filtered_group.loc[filtered_group['board'] == 0]['cal'].mode()[0]+3,  100, 500,  0, 600],
    1: [filtered_group.loc[filtered_group['board'] == 1]['cal'].mode()[0]-3, filtered_group.loc[filtered_group['board'] == 1]['cal'].mode()[0]+3,   0,  1100, 0, 600], # pixel (15, 6), (15, 7) Sep 28th data
    2: [filtered_group.loc[filtered_group['board'] == 2]['cal'].mode()[0]-3, filtered_group.loc[filtered_group['board'] == 2]['cal'].mode()[0]+3,   0,  1100, 0, 600],
    # 3: [filtered_group.loc[filtered_group['board'] == 3]['cal'].mode()[0]-3, filtered_group.loc[filtered_group['board'] == 3]['cal'].mode()[0]+3,   0,  1100, 0, 600],
}
# tdc_cuts = {
#     # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
#     0: [200, 210,   0, 1100,   0, 600],
#     # 1: [173, 180, 250,  325,   0, 600], # pixel (15, 8) Sep 28th data
#     # 1: [173, 180, 262,  337,   0, 600], # pixel (15, 9) Sep 28th data
#     1: [173, 180, 100,  500,   0, 600],
#     2: [160, 220,   0, 1100,   0, 600],
#     3: [189, 195,   0, 1000,   0, 600], # pixel ()
# }

tdc_filtered_df = helper.tdc_event_selection(filtered_group, tdc_cuts)
tdc_filtered_df = helper.singlehit_event_clear(tdc_filtered_df)
tdc_filtered_df.info()

In [ ]:
del filtered_group

In [ ]:
h_tdc_selection = helper.return_hist(tdc_filtered_df, chip_names, chip_labels, hist_bins=[100, 128, 256])

In [ ]:
helper.plot_1d_TDC_histograms(h_tdc_selection, chip_names[0], chip_fignames[0], chip_figtitles[0],
                                fig_path, save=False, show=True, tag="after_tdc_cut",
                                title_tag=f", Pixel ({pix_dict[0][0]}, {pix_dict[0][1]}) after TDC cut", slide_friendly=True)

In [ ]:
helper.plot_1d_TDC_histograms(h_tdc_selection, chip_names[1], chip_fignames[1], chip_figtitles[1],
                                fig_path, save=False, show=True, tag="",
                                title_tag=f", Pixel ({pix_dict[1][0]}, {pix_dict[1][1]}) after TDC cut", slide_friendly=True)

In [ ]:
helper.plot_1d_TDC_histograms(h_tdc_selection, chip_names[2], chip_fignames[2], chip_figtitles[2],
                                fig_path, save=False, show=True, tag="",
                                title_tag=f", Pixel ({pix_dict[2][0]}, {pix_dict[2][1]}) after TDC cut", slide_friendly=True)

In [ ]:
helper.plot_1d_TDC_histograms(h_tdc_selection, chip_names[3], chip_fignames[3], chip_figtitles[3],
                                fig_path, save=False, show=True, tag="",
                                title_tag=f", Pixel ({pix_dict[3][0]}, {pix_dict[3][1]}) after TDC cut", slide_friendly=True)

In [ ]:
del h_tdc_selection

## Convert TDC code to TDC time in [ns]

In [ ]:
tdc_filtered_df = tdc_filtered_df.pivot(index=["evt"], columns=["board"], values=["row", "col", "toa", "tot", "cal"]).reset_index()

In [ ]:
board_to_analyze = [0, 1, 2]

In [ ]:
d = {
    'evt': tdc_filtered_df['evt'].unique(),
}

for idx in board_to_analyze:
    bins = 3.125/tdc_filtered_df['cal'][idx].mean()
    d[f'toa_b{str(idx)}'] = 12.5 - tdc_filtered_df['toa'][idx] * bins
    d[f'tot_b{str(idx)}'] = (2*tdc_filtered_df['tot'][idx] - np.floor(tdc_filtered_df['tot'][idx]/32)) * bins

df_in_time = pd.DataFrame(data=d)
del d, tdc_filtered_df

corr_toas = helper.three_board_iterative_timewalk_correction(df_in_time, 5, 3, board_list=board_to_analyze)

## Traditional Time Walk Correction

In [ ]:
del_toa_b0 = (0.5*(df_in_time['toa_b1'] + df_in_time['toa_b2']) - df_in_time['toa_b0']).values
del_toa_b1 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b2']) - df_in_time['toa_b1']).values
del_toa_b3 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b1']) - df_in_time['toa_b2']).values

In [ ]:
coeff_b0 = np.polyfit(df_in_time['tot_b0'].values, del_toa_b0, 3)
poly_func_b0 = np.poly1d(coeff_b0)

coeff_b1 = np.polyfit(df_in_time['tot_b1'].values, del_toa_b1, 3)
poly_func_b1 = np.poly1d(coeff_b1)

coeff_b3 = np.polyfit(df_in_time['tot_b2'].values, del_toa_b3, 3)
poly_func_b3 = np.poly1d(coeff_b3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b2'].values,  del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b2'].values, poly_func_b3(df_in_time['tot_b2'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
corr_toas = helper.three_board_iterative_timewalk_correction(df_in_time, 5, 3, board_list=[0, 1, 3])

In [ ]:
corr_toas

In [ ]:
nth_del_toa_b0 = (0.5*(corr_toas['toa_b1'] + corr_toas['toa_b2']) - corr_toas['toa_b0'])
nth_del_toa_b1 = (0.5*(corr_toas['toa_b0'] + corr_toas['toa_b2']) - corr_toas['toa_b1'])
nth_del_toa_b3 = (0.5*(corr_toas['toa_b0'] + corr_toas['toa_b1']) - corr_toas['toa_b2'])

In [ ]:
coeff_b0 = np.polyfit(df_in_time['tot_b0'].values, nth_del_toa_b0, 3)
poly_func_b0 = np.poly1d(coeff_b0)

coeff_b1 = np.polyfit(df_in_time['tot_b1'].values, nth_del_toa_b1, 3)
poly_func_b1 = np.poly1d(coeff_b1)

coeff_b3 = np.polyfit(df_in_time['tot_b2'].values, nth_del_toa_b3, 3)
poly_func_b3 = np.poly1d(coeff_b3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  nth_del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  nth_del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b2'].values,  nth_del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b2'].values, poly_func_b3(df_in_time['tot_b2'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
corrTOA_b0 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
corrTOA_b1 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
corrTOA_b3 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

corrTOA_b0.fill(corr_toas[0])
corrTOA_b1.fill(corr_toas[1])
corrTOA_b3.fill(corr_toas[2])

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b0, ax=axes[0], lw=2)
axes[0].set_xlabel('Time Walk Corrected TOA [ns]')
axes[0].set_ylabel('Entries')

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b1, ax=axes[1], lw=2)
axes[1].set_xlabel('Time Walk Corrected TOA [ns]')
axes[1].set_ylabel('Entries')

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b3, ax=axes[2], lw=2)
axes[2].set_xlabel('Time Walk Corrected TOA [ns]')
axes[2].set_ylabel('Entries')

plt.tight_layout()

In [ ]:
tmp_dict = {
    'evt': df_in_time['evt'].values,
    'corr_toa_b0': corr_toas[0],
    'corr_toa_b1': corr_toas[1],
    'corr_toa_b3': corr_toas[2],
}

df_in_time_corr = pd.DataFrame(tmp_dict)
del tmp_dict
df_in_time_corr

In [ ]:
diff_b01 = df_in_time_corr['corr_toa_b0'] - df_in_time_corr['corr_toa_b1']
diff_b03 = df_in_time_corr['corr_toa_b0'] - df_in_time_corr['corr_toa_b3']
diff_b13 = df_in_time_corr['corr_toa_b1'] - df_in_time_corr['corr_toa_b3']

dTOA_b01 = hist.Hist(hist.axis.Regular(80, diff_b01.mean().round(2)-0.8, diff_b01.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_b03 = hist.Hist(hist.axis.Regular(80, diff_b03.mean().round(2)-0.8, diff_b03.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_b13 = hist.Hist(hist.axis.Regular(80, diff_b13.mean().round(2)-0.8, diff_b13.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

dTOA_b01.fill(diff_b01)
dTOA_b03.fill(diff_b03)
dTOA_b13.fill(diff_b13)

### Fit using lmfit GaussianModel

In [ ]:
fit_params_lmfit = []
params = helper.lmfit_gaussfit_with_pulls(diff_b01, dTOA_b01, std_range_cut=0.4, width_factor=1.25, fig_title='Board 0 - Board 1',
                                          use_pred_uncert=True, no_show_fit=False, no_draw=False, get_chisqure=False)
fit_params_lmfit.append(params)
params = helper.lmfit_gaussfit_with_pulls(diff_b03, dTOA_b03, std_range_cut=0.4, width_factor=1.25, fig_title='Board 0 - Board 3',
                                          use_pred_uncert=True, no_show_fit=False, no_draw=False, get_chisqure=False)
fit_params_lmfit.append(params)
params = helper.lmfit_gaussfit_with_pulls(diff_b13, dTOA_b13, std_range_cut=0.4, width_factor=1.25, fig_title='Board 1 - Board 3',
                                          use_pred_uncert=True, no_show_fit=False, no_draw=False, get_chisqure=False)
fit_params_lmfit.append(params)

In [ ]:
res_b0,err_b0 = helper.return_resolution_three_board(fit_params_lmfit[0][0], fit_params_lmfit[0][1],
                                            fit_params_lmfit[1][0], fit_params_lmfit[1][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1])
res_b1,err_b1 = helper.return_resolution_three_board(fit_params_lmfit[0][0], fit_params_lmfit[0][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1],
                                            fit_params_lmfit[1][0], fit_params_lmfit[1][1])
res_b3,err_b3 = helper.return_resolution_three_board(fit_params_lmfit[1][0], fit_params_lmfit[1][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1],
                                            fit_params_lmfit[0][0], fit_params_lmfit[0][1])

print(f'Board 0: {res_b0:.2f} ps, error: {err_b0:.2f} ps')
print(f'Board 1: {res_b1:.2f} ps, error: {err_b1:.2f} ps')
print(f'Board 3: {res_b3:.2f} ps, error: {err_b3:.2f} ps')

### Multi-polynomial fit

In [ ]:
def poly2D_2_fit(M, *args):
    x, y = M
    return helper.poly2D(2, x, y, *args)

def poly2D_3_fit(M, *args):
    x, y = M
    return helper.poly2D(3, x, y, *args)

def poly2D_4_fit(M, *args):
    x, y = M
    return helper.poly2D(4, x, y, *args)

def poly3D_2_fit(M, *args):
    x, y, z = M
    return helper.poly3D(2, x, y, z, *args)

helper.poly2D(0, df_in_time['tot_b0'], df_in_time['tot_b1'], 2)                              #  pol: z = 2
helper.poly2D(1, df_in_time['tot_b0'], df_in_time['tot_b1'], 2, 4, 1)                        #  pol: z = 2 + 4x + y
helper.poly2D(2, df_in_time['tot_b0'], df_in_time['tot_b1'], 2, 0.2, -0.9, 0.3, -0.1, 0.01)  #  pol: z = 2 + 0.2x - 0.9x^2 + 0.3y - 0.1xy + 0.01y^2

import scipy.optimize as opt
x = df_in_time['tot_b0']
y = df_in_time['tot_b1']
z = df_in_time['tot_b3']
data = df_in_time['toa_b0'] - df_in_time['toa_b1']
data = df_in_time['toa_b0'] - df_in_time['toa_b3']
data = df_in_time['toa_b1'] - df_in_time['toa_b3']
initial_guess = (data.mean(), 0, 0, 0, 0, 0)
popt, pcov = opt.curve_fit(poly2D_2_fit, (x,y), data, p0 = initial_guess)
corr_data = helper.poly2D(2, df_in_time['tot_b0'], df_in_time['tot_b1'], *popt)

initial_guess = (data.mean(), 0, 0, 0, 0, 0, 0, 0, 0, 0)
popt, pcov = opt.curve_fit(poly2D_3_fit, (x,y), data, p0 = initial_guess)
corr_data = helper.poly2D(3, df_in_time['tot_b0'], df_in_time['tot_b1'], *popt)

initial_guess = (data.mean(), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
popt, pcov = opt.curve_fit(poly2D_4_fit, (x,y), data, p0 = initial_guess)
corr_data = helper.poly2D(4, df_in_time['tot_b0'], df_in_time['tot_b1'], *popt)

#initial_guess = (data.mean(), 0, 0, 0, 0, 0, 0, 0, 0, 0)
#popt, pcov = opt.curve_fit(poly3D_2_fit, (x,y,z), data, p0 = initial_guess)
#corr_data = poly3D(2, df_in_time['tot_b0'], df_in_time['tot_b1'], df_in_time['tot_b3'], *popt)

dTOA_test = hist.Hist(hist.axis.Regular(80, corr_data.mean().round(2)-0.8, corr_data.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_test.fill(corr_data)
dTOA_test

params = helper.lmfit_gaussfit_with_pulls(corr_data, dTOA_test, std_range_cut=0.4, width_factor=1, fig_title='Board 1 - Board 3', use_pred_uncert=True)

## Pairwise Time Walk Correction -  Pairwise Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input

In [ ]:
def return_dense_model(numpars=2, print_summary=False):
    input  = Input(shape=(numpars,), name='input')
    dense1 = Dense(8, activation='relu', name='dense1',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(input)
    dense2 = Dense(8, activation='relu', name='dense2',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense1)
    output = Dense(1, activation='linear', name='output',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense2)
    model  = Model(inputs=[input], outputs=output, name="simple_dense_NN")
    model.compile(loss='mse', optimizer='adam')
    if(print_summary): print(model.summary())
    return model

model = return_dense_model(print_summary=True)
del model

## --------------------------------------------------------------

def return_combined_dense_model(numpars=3, print_summary=False):
    input  = Input(shape=(numpars,), name='input')
    dense1 = Dense(8, activation='relu', name='dense1',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(input)
    dense2 = Dense(8, activation='relu', name='dense2',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense1)
    output = Dense(3, activation='linear', name='output',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense2)
    model  = Model(inputs=[input], outputs=output, name="simple_dense_NN")
    model.compile(loss='mse', optimizer='adam')
    if(print_summary): print(model.summary())
    return model

model = return_combined_dense_model(print_summary=True)
del model

In [ ]:
def etroc_regression_using_NN(
        input_df: pd.DataFrame,
        variables: list,
        data_tag: str,
        extra_tag: str,
        board_tag: str,
        ensemble_count: int,
        figure_title: str,
        do_plotting: bool,
        epochs: int = 300,
    ):
    filename = f'{data_tag}_weights_{extra_tag}_{board_tag}'

    for en_idx in range(ensemble_count):
        model = return_dense_model(numpars=2)
        checkpointer = ModelCheckpoint(f'models/NNRun{en_idx}_{filename}.hdf5', verbose=0, save_best_only=True, monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)

        shuffled_df = input_df.sample(frac=1)

        history = model.fit(
            shuffled_df[[variables[0], variables[1]]].values,
            (shuffled_df[variables[2]]-shuffled_df[variables[3]]).values,
            validation_split=0.3,
            epochs=epochs,
            callbacks=[checkpointer,term,escb],
            verbose=0)

        del model

        if (do_plotting):
            #plot the loss and validation loss of the dataset
            fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
            hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
            ax.set_title(f'Model {en_idx}, {figure_title}', loc="right", size=25)
            plt.plot(history.history['loss'], label='Train data')
            plt.plot(history.history['val_loss'], label='Validation data')
            plt.ylabel('Loss (MSE)')
            plt.yscale("log")
            plt.legend()
            plt.savefig(f"models/NNRun{en_idx}_{filename}.png")
            plt.show()

## --------------------------------------------------------------

def etroc_regression_using_combined_NN(
        input_df: pd.DataFrame,
        variables: list,
        data_tag: str,
        extra_tag: str,
        ensemble_count: int,
        figure_title: str,
        do_plotting: bool,
        epochs: int = 300,
    ):
    filename = f'{data_tag}_weights_{extra_tag}'

    inner_df = input_df.copy()
    inner_df['delta01'] = inner_df[variables[3]] - inner_df[variables[4]]
    inner_df['delta03'] = inner_df[variables[3]] - inner_df[variables[5]]
    inner_df['delta13'] = inner_df[variables[4]] - inner_df[variables[5]]

    for en_idx in range(ensemble_count):
        model = return_combined_dense_model(numpars=3)
        checkpointer = ModelCheckpoint(f'models/CombinedNNRun{en_idx}_{filename}.hdf5', verbose=0, save_best_only=True, monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)

        shuffled_df = inner_df.sample(frac=1)

        history = model.fit(
            shuffled_df[[variables[0], variables[1], variables[2]]].values,
            shuffled_df[['delta01', 'delta03', 'delta13']].values,
            validation_split=0.3,
            epochs=epochs,
            callbacks=[checkpointer,term,escb],
            verbose=0)

        del model

        if (do_plotting):
            #plot the loss and validation loss of the dataset
            fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
            hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
            ax.set_title(f'Model {en_idx}, {figure_title}', loc="right", size=25)
            plt.plot(history.history['loss'], label='Train data')
            plt.plot(history.history['val_loss'], label='Validation data')
            plt.ylabel('Loss (MSE)')
            plt.yscale("log")
            plt.legend()
            plt.savefig(f"models/CombinedNNRun{en_idx}_{filename}.png")
            plt.show()

In [ ]:
ensemble_count = 10
data_tag = 'tb_sep28_offset24_Bottom_offset6'

# extra_tag = "WB156_CALNarrow_TOA275to350forWB_NoTOTcut"
extra_tag = "WB157_CALNarrow_TOA275to350forWB_NoTOTcut"
# extra_tag = "WB158_CALNarrow_TOA250to325forWB_NoTOTcut"
# extra_tag = "WB159_CALNarrow_TOA262to337forWB_NoTOTcut"

# extra_tag = "WB156_CALNarrow_TOA100to500forWB_NoTOTcut"
# extra_tag = "WB157_CALNarrow_TOA100to500forWB_NoTOTcut"
# extra_tag = "WB158_CALNarrow_TOA100to500forWB_NoTOTcut"
# extra_tag = "WB159_CALNarrow_TOA100to500forWB_NoTOTcut"

### Run NN Training (skip if you don't want to do training)

In [ ]:
vars = ['tot_b0', 'tot_b1', 'toa_b0', 'toa_b1']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b01", ensemble_count, figure_title='Board 0 - Board 1', do_plotting=True)

vars = ['tot_b0', 'tot_b3', 'toa_b0', 'toa_b3']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b03", ensemble_count, figure_title='Board 0 - Board 3', do_plotting=True)

vars = ['tot_b1', 'tot_b3', 'toa_b1', 'toa_b3']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b13", ensemble_count, figure_title='Board 1 - Board 3', do_plotting=True)

In [ ]:
vars = ['tot_b0', 'tot_b1', 'tot_b3', 'toa_b0', 'toa_b1', 'toa_b3']
etroc_regression_using_combined_NN(df_in_time, vars, data_tag, extra_tag, ensemble_count, figure_title='Combined Loss', do_plotting=True)

### Load individual trained models

In [ ]:
model_b01 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b01'
for en_idx in range(ensemble_count):
    model_b01.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
    else: Y_pred += model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
del model_b01
Y_pred_b01 = Y_pred/ensemble_count

model_b03 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b03'
for en_idx in range(ensemble_count):
    model_b03.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
del model_b03
Y_pred_b03 = Y_pred/ensemble_count

model_b13 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b13'
for en_idx in range(ensemble_count):
    model_b13.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b13.predict(df_in_time[['tot_b1',  'tot_b3']].values, verbose=0).flatten()
del model_b13
Y_pred_b13 = Y_pred/ensemble_count

In [ ]:
data_b01 = (df_in_time['toa_b0']-df_in_time['toa_b1']).values-Y_pred_b01
data_b03 = (df_in_time['toa_b0']-df_in_time['toa_b3']).values-Y_pred_b03
data_b13 = (df_in_time['toa_b1']-df_in_time['toa_b3']).values-Y_pred_b13

dTOA_NN_b01 = hist.Hist(hist.axis.Regular(50, data_b01.mean().round(2)-0.5, data_b01.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b03 = hist.Hist(hist.axis.Regular(50, data_b03.mean().round(2)-0.5, data_b03.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b13 = hist.Hist(hist.axis.Regular(50, data_b13.mean().round(2)-0.5, data_b13.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

dTOA_NN_b01.fill(data_b01)
dTOA_NN_b03.fill(data_b03)
dTOA_NN_b13.fill(data_b13)

### Load combined trained models

In [ ]:
model_combined = return_combined_dense_model(numpars=3)
filename = f'{data_tag}_weights_{extra_tag}'
for en_idx in range(ensemble_count):
    model_combined.load_weights(f'models/CombinedNNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_combined.predict(df_in_time[['tot_b0', 'tot_b1', 'tot_b3']].values, verbose=0)
    else: Y_pred += model_combined.predict(df_in_time[['tot_b0', 'tot_b1', 'tot_b3']].values, verbose=0)
del model_combined
Y_pred_combined = Y_pred/ensemble_count

In [ ]:
data_b01_combined = (df_in_time['toa_b0']-df_in_time['toa_b1']).values-Y_pred_combined[:,0]
data_b03_combined = (df_in_time['toa_b0']-df_in_time['toa_b3']).values-Y_pred_combined[:,1]
data_b13_combined = (df_in_time['toa_b1']-df_in_time['toa_b3']).values-Y_pred_combined[:,2]

dTOA_NN_b01_combined = hist.Hist(hist.axis.Regular(50, data_b01_combined.mean().round(2)-0.5, data_b01_combined.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b03_combined = hist.Hist(hist.axis.Regular(50, data_b03_combined.mean().round(2)-0.5, data_b03_combined.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b13_combined = hist.Hist(hist.axis.Regular(50, data_b13_combined.mean().round(2)-0.5, data_b13_combined.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

dTOA_NN_b01_combined.fill(data_b01_combined)
dTOA_NN_b03_combined.fill(data_b03_combined)
dTOA_NN_b13_combined.fill(data_b13_combined)

### Fit using lmfit GaussianModel with NN outputs

In [ ]:
fit_params_lmfit_NN = []
params = helper.lmfit_gaussfit_with_pulls(data_b01, dTOA_NN_b01, std_range_cut=0.4, width_factor=1.25, fig_title='Board 0 - Board 1', use_pred_uncert=True)
fit_params_lmfit_NN.append(params)
params = helper.lmfit_gaussfit_with_pulls(data_b03, dTOA_NN_b03, std_range_cut=0.4, width_factor=1.25, fig_title='Board 0 - Board 3', use_pred_uncert=True)
fit_params_lmfit_NN.append(params)
params = helper.lmfit_gaussfit_with_pulls(data_b13, dTOA_NN_b13, std_range_cut=0.4, width_factor=1.25, fig_title='Board 1 - Board 3', use_pred_uncert=True)
fit_params_lmfit_NN.append(params)

In [ ]:
fit_params_lmfit_NN_combined = []
params = helper.lmfit_gaussfit_with_pulls(data_b01_combined, dTOA_NN_b01_combined, std_range_cut=0.4, width_factor=1, fig_title='Board 0 - Board 1', use_pred_uncert=True)
fit_params_lmfit_NN_combined.append(params)
params = helper.lmfit_gaussfit_with_pulls(data_b03_combined, dTOA_NN_b03_combined, std_range_cut=0.4, width_factor=1, fig_title='Board 0 - Board 3', use_pred_uncert=True)
fit_params_lmfit_NN_combined.append(params)
params = helper.lmfit_gaussfit_with_pulls(data_b13_combined, dTOA_NN_b13_combined, std_range_cut=0.4, width_factor=1, fig_title='Board 1 - Board 3', use_pred_uncert=True)
fit_params_lmfit_NN_combined.append(params)

In [ ]:
res_b0_NN,err_b0_NN = helper.return_resolution_ps(fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1],
                                            fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1])
res_b1_NN,err_b1_NN = helper.return_resolution_ps(fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1],
                                            fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1])
res_b3_NN,err_b3_NN = helper.return_resolution_ps(fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1],
                                            fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1])

print(f'Board 0: {res_b0_NN:.2f} ps, error: {err_b0_NN:.2f} ps')
print(f'Board 1: {res_b1_NN:.2f} ps, error: {err_b1_NN:.2f} ps')
print(f'Board 3: {res_b3_NN:.2f} ps, error: {err_b3_NN:.2f} ps')

In [ ]:
res_b0_NN_combined,err_b0_NN_combined = helper.return_resolution_ps(fit_params_lmfit_NN_combined[0][0], fit_params_lmfit_NN_combined[0][1],
                                            fit_params_lmfit_NN_combined[1][0], fit_params_lmfit_NN_combined[1][1],
                                            fit_params_lmfit_NN_combined[2][0], fit_params_lmfit_NN_combined[2][1])
res_b1_NN_combined,err_b1_NN_combined = helper.return_resolution_ps(fit_params_lmfit_NN_combined[0][0], fit_params_lmfit_NN_combined[0][1],
                                            fit_params_lmfit_NN_combined[2][0], fit_params_lmfit_NN_combined[2][1],
                                            fit_params_lmfit_NN_combined[1][0], fit_params_lmfit_NN_combined[1][1])
res_b3_NN_combined,err_b3_NN_combined = helper.return_resolution_ps(fit_params_lmfit_NN_combined[1][0], fit_params_lmfit_NN_combined[1][1],
                                            fit_params_lmfit_NN_combined[2][0], fit_params_lmfit_NN_combined[2][1],
                                            fit_params_lmfit_NN_combined[0][0], fit_params_lmfit_NN_combined[0][1])

print(f'Board 0: {res_b0_NN_combined:.2f} ps, error: {err_b0_NN_combined:.2f} ps')
print(f'Board 1: {res_b1_NN_combined:.2f} ps, error: {err_b1_NN_combined:.2f} ps')
print(f'Board 3: {res_b3_NN_combined:.2f} ps, error: {err_b3_NN_combined:.2f} ps')

## Make final result plot

In [ ]:
names = ['Top', 'Middle', 'Bottom']

board_resols = {
    'Single board TWC': ((res_b1, res_b0, res_b3), (err_b1, err_b0, err_b3)),
    'Pairwise TWC': ((res_b1_NN, res_b0_NN, res_b3_NN), (err_b1_NN, err_b0_NN, err_b3_NN)),
    # 'Combined TWC': ((res_b1_NN_combined, res_b0_NN_combined, res_b3_NN_combined), (err_b1_NN_combined, err_b0_NN_combined, err_b3_NN_combined)),
}

x = np.arange(len(names))  # the label locations
width = 0.1  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(13, 6), layout='constrained')

for attribute, measurement in board_resols.items():
    offset = width * multiplier
    rects = ax.barh(x + offset, measurement[0], width, edgecolor='white', xerr=measurement[1], label=attribute)
    multiplier += 1

rectangle = mpatch.Rectangle((55, 0.2), 40, 0.6, edgecolor='black', facecolor="none", linewidth=1.5)
ax.add_patch(rectangle)
rx, ry = rectangle.get_xy()
cx = rx + rectangle.get_width()/30.
cy = ry + rectangle.get_height()/2.

text_for_top = 'Top: FFF corner, Wire bonded 2x2, W36-IP7-13 (HPK, split3), offset 24, HV=210V'
text_for_mid = 'Middle (Trigger): FFF corner, Bump bonded 16x16, W13 4-5 (FBK), offset 24, HV=210V'
text_for_bot = 'Bottom: FFF corner, Bump bonded 16x16, W16 P6 (HPK), offset 6, HV=210V'

ax.annotate(f"{text_for_top} \n {text_for_mid} \n {text_for_bot}", (cx, cy),
            color='black', weight='bold', fontsize=11, ha='left', va='center')

# Add some text for labels, title and custom x-axis tick labels, etc.
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(rf'TOA$_{{{names[0]}}} \in \left[{tdc_cuts[1][2]}, {tdc_cuts[1][3]}\right]$', loc="right", size=20)
ax.set_xlabel('Time Resolution [ps]', fontsize=20)
ax.set_yticks(x + width, names)
ax.legend(loc='lower right', fontsize=18)
ax.set_xlim(30, 100)
ax.invert_yaxis()